<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Danish/FastText_biGRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Danish Language

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Danish/

/content/drive/My Drive/dataset/OffenseEval2020/data/Danish


In [3]:
ls

cc.da.300.vec  cc.da.300.vec.gz  offenseval-da-training-v1.tsv


In [0]:
import pandas as pd
import numpy as np
train=pd.read_csv( 'offenseval-da-training-v1.tsv',sep="\t")

In [5]:
train.head()

,id,tweet,subtask_a
0,3131,"Jeg tror det vil være dejlig køligt, men jeg v...",NOT
1,711,Så kommer de nok til at investere i en ny cyke...,NOT
2,2500,Nu er det jo også de Ikea-aber der har lavet s...,OFF
3,2678,"128 Varme emails, er vi enige om at det er sex...",NOT
4,784,"Desværre tyder det på, at amerikanerne er helt...",NOT


In [6]:
train.tail()

,id,tweet,subtask_a
2956,1226,"Ind og ruske tremmer med hende,Den syge kælling!!",OFF
2957,2596,fedtmule,NOT
2958,1802,##HAR I HØRT DET?,NOT
2959,2809,Kommer det bag på nogen? Det er jo fucking var...,OFF
2960,4000 [deleted] NOT,NaN,NaN


In [0]:
train.drop(train.tail(n=1).index,inplace=True)

In [0]:
train=train.sample(frac=1)

In [0]:
labels=train['subtask_a']
train=train['tweet']
labels=pd.factorize(labels)[0]

In [10]:
import collections
collections.Counter(labels)

Counter({0: 384, 1: 2576})

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

Using TensorFlow backend.


In [12]:
len(sequences_matrix),len(labels)

(2960, 2960)

In [0]:
from keras.models import Sequential, Model
from keras.layers import Embedding,CuDNNGRU,CuDNNLSTM,GRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D,concatenate,Input
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
from keras import backend as K
def f1(y_true, y_pred):
    '''
    metric from here 
    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    '''
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [15]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

9970


In [0]:
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.da.300.vec.gz

In [0]:
# import gzip
# import shutil
# with gzip.open('cc.da.300.vec.gz', 'rb') as f_in:
#     with open('cc.da.300.vec', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [0]:
embed_size=300
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [0]:
embeddings=build_matrix('cc.da.300.vec', word_index)


In [20]:
embeddings.shape

(9970, 300)

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(sequences_matrix,labels,test_size=0.33,stratify=labels,shuffle=True)

In [0]:
inp=Input(shape=(max_len, ))
x=Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
#x=Embedding(input_dim=max_words,output_dim=300,input_length=max_len)(inp)
x = SpatialDropout1D(0.2)(x)
x=Bidirectional(CuDNNLSTM(150, return_sequences = True))(x)
x=Bidirectional(CuDNNLSTM(100, return_sequences = True))(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPool1D()(x)
conc = concatenate([avg_pool, max_pool])
x=Dropout(0.1)(conc)
out=Dense(64, activation="relu")(x)
out=Dense(32, activation="relu")(x)

out=Dense(1, activation="sigmoid")(x)

model=Model(inp,out)
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=[f1])

In [0]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1)

In [25]:


model.fit(x_train,y_train,batch_size=256,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val,y_val),callbacks=[reduce_lr])

Train on 1983 samples, validate on 977 samples
Epoch 1/10
 - 15s - loss: 0.6921 - f1: 0.5332 - val_loss: 0.6605 - val_f1: 0.8595
Epoch 2/10
 - 2s - loss: 0.6846 - f1: 0.6944 - val_loss: 0.6234 - val_f1: 0.9292
Epoch 3/10
 - 2s - loss: 0.6564 - f1: 0.8014 - val_loss: 0.6849 - val_f1: 0.7370
Epoch 4/10
 - 2s - loss: 0.6323 - f1: 0.7241 - val_loss: 0.6311 - val_f1: 0.8476

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 5/10
 - 2s - loss: 0.6367 - f1: 0.7666 - val_loss: 0.5979 - val_f1: 0.8513
Epoch 6/10
 - 2s - loss: 0.5573 - f1: 0.8669 - val_loss: 0.5917 - val_f1: 0.8481
Epoch 7/10
 - 2s - loss: 0.5506 - f1: 0.8673 - val_loss: 0.5564 - val_f1: 0.8642
Epoch 8/10
 - 2s - loss: 0.5373 - f1: 0.8671 - val_loss: 0.5772 - val_f1: 0.8411
Epoch 9/10
 - 2s - loss: 0.5263 - f1: 0.8624 - val_loss: 0.6101 - val_f1: 0.8190

Epoch 00009: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
Epoch 10/10
 - 2s - loss: 0.5221 - f1: 0.8339 - val_loss: 0

In [26]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_val, batch_size=128, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_val, y_pred))

977/977 [==============================] - 1s 546us/step
              precision    recall  f1-score   support

           0       0.29      0.59      0.39       127
           1       0.93      0.79      0.85       850

    accuracy                           0.76       977
   macro avg       0.61      0.69      0.62       977
weighted avg       0.85      0.76      0.79       977



In [0]:
import keras
keras.backend.clear_session()